In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pprint
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support import expected_conditions as EC


import demjson3
import os
import json

import sys
sys.path.append('/full/path/to/web-scraping')



from salaries_functions import create_country_file, write_jobs_to_txt, scrape_pages, parse_response_to_csv, get_netherlands_url, get_dataframe

os.chdir('../database')
from crud_functions import write_dataframes_to_db
from models import Software_Engineer

os.chdir('../salaries')

In [ ]:
netherlands_url = get_netherlands_url()
response = requests.get(netherlands_url)

In [ ]:
germany_url = 'https://techpays.com/europe/germany'
response = requests.get(germany_url)

In [ ]:
netherlands_url = 'https://techpays.com/europe/netherlands'
response = requests.get(netherlands_url)
create_country_file('netherlands')
create_country_file('germany')

In [ ]:
scrape_pages(netherlands_url)
scrape_pages(germany_url)

earlier netherlands iteration crashed at 129th iteration

berlin search has to stop at the 107th iteration! (added break statement)

In [ ]:
parse_response_to_csv('berlin')
parse_response_to_csv('netherlands')

In [ ]:
dutch_avg = dutch_salaries['baseSalaryNumber'].mean() / 12
print('dutch_avg',dutch_avg)

berin_avg = berlin_salaries['baseSalaryNumber'].mean() / 12
print('berin_avg',berin_avg)


In [ ]:
for title in dutch_salaries['title'].dropna().unique():
    if 'senior' in title:
        print(title)
    if 'medior' in title:
        print(title)
    if 'junior' in title:
        print(title)

In [ ]:

def get_page_filter_button_types_dict():
    arguments = dict.fromkeys(('locationFilterButton','jobFamilyFilterButton', 'seniorityFilterButton','roleTypeFilterButton','companyFilterButton'))
    return arguments


def get_filter_button_values(driver, filter_button):
    '''
    This functions receives a WebDriver object and a button id Object
    It returns a dictionary of all possible values for that button.
    '''
    options = {}
    button_class = re.sub(r'Button', '', filter_button) + 'Options'
    print(f'Filter button is {filter_button}, and button class is {button_class}')

    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, filter_button))
    )
    
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    
    
    
    button.click()


    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, button_class))
    )

    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    dropdown = soup.find("div", id= button_class)
    locations = dropdown.find_all("a")

    for loc in locations:
        name = loc.text.strip()
        link = "https://techpays.com" + loc["href"]
        #print(f"{name}: {link}")
        options[name] = link
    button.send_keys(Keys.SPACE)
    return options





#########   MAIN    ####
driver = webdriver.Chrome()
driver.get(netherlands_url)
button_ids = get_page_filter_button_types_dict()
button_id_values = {button_id: {} for button_id in button_ids}

for button_id in button_ids:
    button_id_values = get_filter_button_values(driver, button_id)
    button_ids[button_id] = button_id_values
    print(button_ids[button_id])

    
    
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    driver.execute_script("document.querySelector('body').click();")
    time.sleep(1) 




In [ ]:
print(json.dumps({button_id: button_ids[button_id]}, indent=2, ensure_ascii=False))


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get('/')
async def read_root():
    return 'Hello World'

In [ ]:

os.chdir('../database')
from models import software_engineer

os.chdir('../salaries')

-
from models import software_engineer


In [ ]:
os.chdir('../salaries')

In [ ]:

os.chdir('../database')
from crud_functions import write_dataframes_to_db
from models import software_engineer

In [4]:
df.head()

,Unnamed: 0,title,guid,specialization,city,companyName,totalCompensation,totalCompensationNumber,totalCompensationDetails,baseSalary,baseSalaryNumber,oldYearForData,otherContext
0,0,Senior Software Engineer,7wlsz77oodzc75obmli2a,Backend,NaN,Delivery Hero,"€120,000",120000,"€100K salary, €20K equity/year","€8,333/month",100000,NaN,NaN
1,1,Ethical Hacker,itlsz42deonrsqzwy81xh,NaN,Berlin (flexible days),🔒 German small-size cybersecurity company,"€55,000",55000,"€50K salary, €5K bonus","€4,167/month",50000,2023.0,NaN
2,2,Cyber Security Expert,9mlsyiqxe7mtq9pr3pu7c,NaN,Berlin (hybrid option),Autobahn GmbH,"€75,000",75000,"€75K salary, €5K benefits","€6,233/month",75000,NaN,NaN
3,3,Software Engineer,5mlt0i4auc5foasyx7evx,MTS,Flexible days,AMD,"€99,500",99500,"€82K salary, €10K bonus, €8K equity/year, €3K ...","€6,833/month",82000,NaN,NaN
4,4,AI Algorithm Architect,bqlt1auexcnhb4vpn4spf,NaN,Hildesheim (flexible days),Bosch,"€110,000",110000,€110K salary,"€9,167/month",110000,2023.0,NaN


In [18]:
df.dtypes

Unnamed: 0                    int64
title                        object
guid                         object
specialization               object
city                         object
companyName                  object
totalCompensation            object
totalCompensationNumber       int64
totalCompensationDetails     object
baseSalary                   object
baseSalaryNumber              int64
oldYearForData              float64
otherContext                 object
dtype: object

In [2]:
df = get_dataframe('berlin')

write_dataframes_to_db(df, Software_Engineer)

Unnamed: 0                    int64
title                        object
guid                         object
specialization               object
city                         object
companyName                  object
totalCompensation           float64
totalCompensationNumber       int64
totalCompensationDetails     object
baseSalary                  float64
baseSalaryNumber              int64
oldYearForData              float64
otherContext                 object
dtype: object


DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type double precision: "Backend"
LINE 1: ...ior Software Engineer ', '7wlsz77oodzc75obmli2a', 'Backend',...
                                                             ^

[SQL: INSERT INTO software_engineer (title, guid, specialization, city, "companyName", "totalCompensation", "totalCompensationNumber", "totalCompensationDetails", "baseSalary", "baseSalaryNumber", "oldYearForData", "otherContext") SELECT p0::VARCHAR, p1::V ... 291768 characters truncated ... p11, sen_counter) ORDER BY sen_counter RETURNING software_engineer.id, software_engineer.id AS id__1]
[parameters: {'specialization__0': 'Backend', 'title__0': 'Senior Software Engineer ', 'guid__0': '7wlsz77oodzc75obmli2a', 'baseSalary__0': nan, 'totalCompensationDetails__0': '€100K salary, €20K equity/year', 'city__0': nan, 'baseSalaryNumber__0': 100000, 'totalCompensationNumber__0': 120000, 'companyName__0': 'Delivery Hero', 'otherContext__0': nan,

In [3]:
df.head()

,Unnamed: 0,title,guid,specialization,city,companyName,totalCompensation,totalCompensationNumber,totalCompensationDetails,baseSalary,baseSalaryNumber,oldYearForData,otherContext
0,0,Senior Software Engineer,7wlsz77oodzc75obmli2a,Backend,NaN,Delivery Hero,NaN,120000,"€100K salary, €20K equity/year",NaN,100000,NaN,NaN
1,1,Ethical Hacker,itlsz42deonrsqzwy81xh,NaN,Berlin (flexible days),🔒 German small-size cybersecurity company,NaN,55000,"€50K salary, €5K bonus",NaN,50000,2023.0,NaN
2,2,Cyber Security Expert,9mlsyiqxe7mtq9pr3pu7c,NaN,Berlin (hybrid option),Autobahn GmbH,NaN,75000,"€75K salary, €5K benefits",NaN,75000,NaN,NaN
3,3,Software Engineer,5mlt0i4auc5foasyx7evx,MTS,Flexible days,AMD,NaN,99500,"€82K salary, €10K bonus, €8K equity/year, €3K ...",NaN,82000,NaN,NaN
4,4,AI Algorithm Architect,bqlt1auexcnhb4vpn4spf,NaN,Hildesheim (flexible days),Bosch,NaN,110000,€110K salary,NaN,110000,2023.0,NaN
